In [1]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.library.parameters import SeasonAll
import pandas as pd 

In [2]:
player_dict = players.get_players()
kobe = [player for player in player_dict if player['full_name'] == 'Kobe Bryant'][0]
kobe_id = kobe['id']

In [3]:
gamelog_kobe = playergamelog.PlayerGameLog(player_id=kobe_id, season = SeasonAll.all)

In [4]:
df_kobe_games_all = gamelog_kobe.get_data_frames()[0]

In [5]:
df_kobe_games_all['OPPONENT'] = df_kobe_games_all['MATCHUP'].apply(lambda x: x.split()[-1])

In [6]:
df_kobe_games_all.shape

(1346, 28)

In [7]:
# clean up opponent abbreviation
remap = {"UTH": "UTA", "SAN": "SAS", "PHO": "PHX",  "NJN": "BKN", "NOK": "NOP", "NOH":"NOP", "SEA": "OKC", "CHH": "CHA", "VAN": "MEM"}
df_kobe_games_all['OPPONENT'].replace(remap, inplace=True)
df_kobe_games_all['OPPONENT'].value_counts()

LAC    69
GSW    67
MEM    67
DEN    66
OKC    65
PHX    64
MIN    64
SAC    64
POR    62
HOU    61
SAS    61
DAL    60
UTA    60
NOP    41
NYK    34
CLE    34
IND    34
DET    33
MIA    33
TOR    33
MIL    32
WAS    32
PHI    32
CHI    31
CHA    31
BOS    31
ORL    29
ATL    29
BKN    27
Name: OPPONENT, dtype: int64

In [8]:
df_kobe_games_all.columns

Index(['SEASON_ID', 'Player_ID', 'Game_ID', 'GAME_DATE', 'MATCHUP', 'WL',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE', 'OPPONENT'],
      dtype='object')

In [9]:
mean_df = df_kobe_games_all.groupby('OPPONENT').agg(
    PTS = ("PTS", 'mean')
    Minutes = ('MIN', 'mean'),
    FGM = ('FGM', 'mean'),
    FGA = ('FGA', 'mean'),
    FG3M = ('FG3M', 'mean'),
    FG3A = ('FG3A', 'mean'),
    FTM = ('FTM', 'mean'),
    FTA = ('FTA', 'mean'),
    ).round(1)


In [10]:
total_df = df_kobe_games_all.groupby('OPPONENT').agg(
    Minutes = ('MIN', 'sum'),
    FGM = ('FGM', 'sum'),
    FGA = ('FGA', 'sum'),
    FG3M = ('FG3M', 'sum'),
    FG3A = ('FG3A', 'sum'),
    FTM = ('FTM', 'sum'),
    FTA = ('FTA', 'sum'),  
    ).round(1)

In [11]:
highlight_df = df_kobe_games_all.groupby('OPPONENT').agg(
    Games = ('Game_ID', 'count'),
    Wins = ('WL', lambda x: (x=='W').sum()),
    Losses = ('WL', lambda x: (x=='L').sum()))

In [12]:
d = {}
mean_json = mean_df.to_json(orient = "index")
total_json = total_df.to_json(orient = "index")
highlight_json = highlight_df.to_json(orient = "index")
import json
d["mean"] = json.loads(mean_json)
d["total"] = json.loads(total_json)
d["highlight"] = json.loads(highlight_json)
from IPython.display import JSON
JSON(d)

<IPython.core.display.JSON object>

In [13]:
pd.DataFrame(d).to_json("../dist/assets/data/game_stats.json")